In [26]:
using CSV, DataFrames
using LinearAlgebra, Random, Statistics
using Convex, SCS
using Plots
using StatsPlots

In [27]:
fname = "datasets/county_dataset.csv"
df = DataFrame(CSV.File(fname));

In [28]:
cols = names(df)
for col in cols
    m = .!isnan.(df[:,col])
    println("$(col) $(sum(.!m))")
end

COUNTY 0
single_parent_household 2
youth_not_in_school 3
sex_pay_inequity 0
race_pay_inequity 956
snap_household 0
snap_vulnerabe_household 0
no_internet_household 0
no_smartphone_household 0
no_car_household 0
no_plumbing_household 0
over_60min_commute 0
wfh_inv 0
biking_commute_inv 0
working_class_workers 0
unemployment_rate 0
renter_occupied 0
median_income_inv 0
Crimes Against Persons Rate 395
Crimes Against Property Rate 395
Crimes Against Society Rate 395
ALR_VALB 9
ALR_VALP 9
lahunv1share 99
lasnap1share 99


In [29]:
(n, m) = size(df[!, cols[2:end]])

(3222, 24)

In [30]:
A = Matrix(df[:, cols[2:end]])
M = isnan.(A)

corr = zeros(m,m)
for i in 1:m
    for j in i:m
        mask = (M[:,i] .+ M[:,j]) .== 0
        x = cor(Vector(df[mask ,cols[1+i]]), Vector(df[mask ,cols[1+j]]))
        corr[i, j] = x
        corr[j, i] = x
    end
end

eigs = eigvals(corr)
nfa = sum(eigs .> 1)
k = nfa

6

In [31]:
MAX_ITERS = 20
residual = zeros(MAX_ITERS);
rng = Xoshiro(42)

X_0 = rand(rng, Float64,(n, k));
X_0 = X_0 ./ (X_0 * ones(k, 1))
Y_0 = rand(rng, Float64,(k, m));
H = ones((n, m))
p = 0
constraint = []

A[M] .= 0
H[M] .= 0

for i = 1:MAX_ITERS
    
    if (i % 2) == 1
        Y = Variable((k,m))
        set_value!(Y, Y_0)
        X = X_0
        #constraint = [Y >= 0]
        constraint = [Y[1,:] >= 0]
        #constraint = []
        obj = sum(huber(A - X*Y.*H, .4)) + 12 * norm(Y[2:k,:], 1) + 6 * norm(Y[2:k,:], 2)
    else
        X = Variable((n, k))
        set_value!(X, X_0)
        Y = Y_0
        #constraint = [X >= 0, X * ones(k, 1) <= 1]
        constraint = [X[:,1] >= 0, opnorm(X, Inf) <= 1]
        #constraint = []
        obj = sum(huber(A - X*Y.*H, .4)) #+ 0.001 * norm(ones(1, n) * X[:,2:end], 1)
    end
    
    p = minimize(obj, constraint)
    solve!(p, SCS.Optimizer; silent = true)

    if p.status != Convex.MOI.OPTIMAL
        println("Failed to Converge")
    end

    residual[i] = p.optval

    println("$i, $(residual[i])")

    if (i % 2) == 1
        Y_0 = Y.value
    else
        X_0 = X.value
    end
end

println()

w = mean(abs.(X_0), dims=1) .* mean(abs.(Y_0), dims=2)'
w = w ./ sum(w)
println(w)

1, 7027.621967192549
2, 4642.249650453583
3, 4548.576417456097
4, 3560.988930876602
5, 3890.3662203236786
6, 3011.662966277964
7, 3430.757807691707
8, 2655.545141184659
9, 3216.891656461602
10, 2505.3226149637117
11, 3119.157776876286
12, 2437.8606997314528
13, 3065.840263891375
14, 2403.076048612845
15, 3034.222465744641
16, 2381.576103899683
17, 3011.8269005680368
18, 2367.5712368717586
19, 2994.4873804349004
20, 2357.7906168055956

[0.5892662009587596 0.06925098801772876 0.07922762834963533 0.10455478342208194 0.08136158310559219 0.07633881614620208]


In [32]:
df_Y = DataFrame(Y_0, cols[2:end])
fname = "weights/county_Y.csv"
CSV.write(fname, df_Y)

"weights/county_Y.csv"

In [33]:
df_X = df[:, ["COUNTY"]]
for i in 1:k
    df_X[:, "$i"] = X_0[:, i]
end
fname = "weights/county_X.csv"
CSV.write(fname, df_X)

"weights/county_X.csv"